In [4]:
import os
import json

dataDir = "../data/"
relationDir = os.path.join(dataDir, "relations")
structureFile = os.path.join(relationDir, "structure_relations.json")

# read file from structureFile
relations = set()
with open(structureFile, "r") as f:
    structureRelations = json.load(f)
    for idx, structureRelation in enumerate(structureRelations):
        if structureRelation[1] == "目录":
            structureRelations[idx][1] = "parent_content"
        elif structureRelation[1] == "前置":
            structureRelations[idx][1] = "prerequisites"
        relations.add(structureRelations[idx][1])


print("# of relations: ", len(relations))
print(relations)

print("# of structure relations: ", len(structureRelations))

# of relations:  4
{'prerequisites', 'parent_content', 'co_presence', 'include'}
# of structure relations:  79277


In [5]:
import networkx as nx

graphDir = os.path.join(dataDir, "graph")


def build_graph(relations):
    graph = nx.DiGraph()
    for relation in relations:
        graph.add_edge(relation[0], relation[2], relation=relation[1])
    return graph


graph = build_graph(structureRelations)
# print # of nodes and edges
print("# of nodes: ", graph.number_of_nodes())
print("# of edges: ", graph.number_of_edges())

# export graph to json file
import json

graphJsonFile = os.path.join(graphDir, "structure_graph.json")
with open(graphJsonFile, "w") as f:
    json.dump(nx.node_link_data(graph), f, ensure_ascii=False, indent=4)

# of nodes:  3864
# of edges:  79277


In [7]:
# find all links connected to the node named "Deep Learning"
links = graph.edges(data=True)
print(len(links))
for link in links:
    if link[2]["relation"] == "include" and link[0] == "4.1 Overflow and Underflow":
        print(link)

79277
('4.1 Overflow and Underflow', 'overflow', {'relation': 'include'})
('4.1 Overflow and Underflow', 'evaluating a mathematical function', {'relation': 'include'})
('4.1 Overflow and Underflow', 'exceptions', {'relation': 'include'})
('4.1 Overflow and Underflow', 'bit patterns', {'relation': 'include'})
('4.1 Overflow and Underflow', 'Overflow and Underflow', {'relation': 'include'})
('4.1 Overflow and Underflow', 'underflow', {'relation': 'include'})
('4.1 Overflow and Underflow', 'low-level libraries', {'relation': 'include'})
('4.1 Overflow and Underflow', 'numerical error', {'relation': 'include'})
('4.1 Overflow and Underflow', 'continuous math', {'relation': 'include'})
('4.1 Overflow and Underflow', 'algorithms', {'relation': 'include'})
('4.1 Overflow and Underflow', 'numerically stable', {'relation': 'include'})
('4.1 Overflow and Underflow', 'optimization', {'relation': 'include'})
('4.1 Overflow and Underflow', 'constant c', {'relation': 'include'})
('4.1 Overflow and U